In [48]:
import config

In [49]:
import spotipy
import json
import pprint
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from pyjarowinkler import distance
from spotipy.oauth2 import SpotifyClientCredentials


#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.c_id,
                                                           client_secret= config.c_se))

### Getting playlists

In [50]:
# locating playlist
playlist_country = sp.user_playlist_tracks("spotify", "3RPaWvjgg1SAi8oIh09Irp",market="GB")
playlist_rock = sp.user_playlist_tracks("spotify", "7KMtxb9kh3TtVvJTdLtJ2K",market="GB")
playlist_house = sp.user_playlist_tracks("spotify", "4VXFwpNvQysB8yS6E2t2ls",market="GB")
playlist_classicals = sp.user_playlist_tracks("spotify", "1h0CEZCm6IbFTbxThn6Xcs",market="GB")
playlist_dance = sp.user_playlist_tracks("spotify", "794tEP1QrKMcaQcFmEyFst",market="GB")
playlist_hiphop = sp.user_playlist_tracks("spotify", "0agbFCQZvK4q9WR0d4Iqcj",market="GB")
playlist_pop = sp.user_playlist_tracks("spotify", "2YRe7HRKNRvXdJBp9nXFza",market="GB")
playlist_jazz = sp.user_playlist_tracks("spotify", "7g2KyiE9RI5F1fV60OIUu4",market="GB")

### Getting tracks

In [92]:
track=[]

In [65]:
# extracting tracks from playlist
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id,market="GB")
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [66]:
tracks_country = get_playlist_tracks("username", "3RPaWvjgg1SAi8oIh09Irp")

In [67]:
for i in range(0, len(tracks_country)):
    track.append(tracks_country[i]['track']['name'])

In [93]:
track

[]

### Getting artists

In [94]:
artist=[]

In [70]:
# extracting artists from track
def get_artists_from_track(track):
    return [artist["name"] for artist in track["artists"]]

In [71]:
# extracting artists from playlist
def get_artists_from_playlist(playlist_id):
    tracks_from_playlist = get_playlist_tracks("spotify", playlist_id)
    return list(get_artists_from_track(track["track"]) for track in tracks_from_playlist)

In [72]:
artists_country = get_artists_from_playlist("3RPaWvjgg1SAi8oIh09Irp")

In [73]:
artist=[]
for i in artists_country:
    artist.append(", ".join(i))

In [95]:
artist

[]

### Getting track ids

In [96]:
track_id=[]

In [76]:
for i in range(0, len(tracks_country)):
    track_id.append(tracks_country[i]['track']['id'])

In [97]:
track_id

[]

In [78]:
print(len(track))
print(len(artist))
print(len(track_id))

747
747
747


### Combining data

In [79]:
# each list becomes a column
track_db = pd.DataFrame({'title':track,
                         'artist':artist,
                         'id':track_id})
track_db.head()

,title,artist,id
0,Crazy - Single Version,"Patsy Cline, The Jordanaires",3zpj9dvJABiyMrmLCPw6i8
1,Folsom Prison Blues,"Johnny Cash, The Tennessee Two",121sjkub4p4zcFNozkKnQD
2,I Will Always Love You,Dolly Parton,6nRvgo4ySezQKD1D7NIqAu
3,Springsteen,Eric Church,05ZACkzW8YbRq3eFgIfSNB
4,He Stopped Loving Her Today,George Jones,6qMgx8ys4MNNxypaIvbG0i


In [98]:
track_df = pd.DataFrame({'title':track,
                     'artist':artist,
                     'id':track_id})

In [103]:
def append_info(playlist_id):
    global track_df
    new_track_list = get_playlist_tracks("spotify", playlist_id)
    new_artist_list = get_artists_from_playlist(playlist_id)
    new_track=[]
    new_artist=[]
    new_track_id=[]
    for i in range(0, len(new_track_list)):
        new_track.append(new_track_list[i]['track']['name'])
    for i in new_artist_list:
        new_artist.append(", ".join(i))
    for i in range(0, len(new_track_list)):
        new_track_id.append(new_track_list[i]['track']['id']) 
    new_df = pd.DataFrame({'title':new_track,
                         'artist':new_artist,
                         'id':new_track_id})
    track_df = track_df.append(new_df)

    

In [83]:
track_df = track_df.append(track_db)

In [110]:
track_df

,title,artist,id
0,Crazy - Single Version,"Patsy Cline, The Jordanaires",3zpj9dvJABiyMrmLCPw6i8
1,Folsom Prison Blues,"Johnny Cash, The Tennessee Two",121sjkub4p4zcFNozkKnQD
2,I Will Always Love You,Dolly Parton,6nRvgo4ySezQKD1D7NIqAu
3,Springsteen,Eric Church,05ZACkzW8YbRq3eFgIfSNB
4,He Stopped Loving Her Today,George Jones,6qMgx8ys4MNNxypaIvbG0i
...,...,...,...
995,Jealousy,Martin Solveig,1u4HCtZ0iI8P8Bb4YcGuYP
996,Rise - Lushington Remix,"Miura Keys, Raphaella, Lushington",2BjeIyXzFtM8s929gzXSvN
997,Make Me Feel,John Summit,4vgP9RNzsXEUcW7Ws50VrI
998,Out Of Control - Pat Vocal Mix,Lupe Fuentes,1QqON1dtjcsi2As0asNDQK


In [105]:
append_info("3RPaWvjgg1SAi8oIh09Irp")

In [107]:
append_info("7KMtxb9kh3TtVvJTdLtJ2K")

In [109]:
append_info("4VXFwpNvQysB8yS6E2t2ls")

In [47]:
append_info("794tEP1QrKMcaQcFmEyFst")

,title,artist,id
0,Crazy - Single Version,"Patsy Cline, The Jordanaires",3zpj9dvJABiyMrmLCPw6i8
1,Folsom Prison Blues,"Johnny Cash, The Tennessee Two",121sjkub4p4zcFNozkKnQD
2,I Will Always Love You,Dolly Parton,6nRvgo4ySezQKD1D7NIqAu
3,Springsteen,Eric Church,05ZACkzW8YbRq3eFgIfSNB
4,He Stopped Loving Her Today,George Jones,6qMgx8ys4MNNxypaIvbG0i
...,...,...,...
93,9Pm (Till I Come),ATB,1CgbwsrNDlFrRuk2ebQ7zr
94,Red Alert - Jaxx Radio Mix,Basement Jaxx,2F5QeVVmCOuelAkRbni1bW
95,Love Is Gone,"David Guetta, Chris Willis",2oWIk5cGvxMsJndbd5aW7g
96,Blue (Da Ba Dee) - Gabry Ponte Ice Pop Radio,"Eiffel 65, Gabry Ponte",2yAVzRiEQooPEJ9SYx11L3


In [ ]:
playlist_rock = sp.user_playlist_tracks("spotify", "7KMtxb9kh3TtVvJTdLtJ2K",market="GB")
playlist_house = sp.user_playlist_tracks("spotify", "4VXFwpNvQysB8yS6E2t2ls",market="GB")
playlist_classicals = sp.user_playlist_tracks("spotify", "1h0CEZCm6IbFTbxThn6Xcs",market="GB")
playlist_dance = sp.user_playlist_tracks("spotify", "794tEP1QrKMcaQcFmEyFst",market="GB")
playlist_hiphop = sp.user_playlist_tracks("spotify", "0agbFCQZvK4q9WR0d4Iqcj",market="GB")
playlist_pop = sp.user_playlist_tracks("spotify", "2YRe7HRKNRvXdJBp9nXFza",market="GB")
playlist_jazz = sp.user_playlist_tracks("spotify", "7g2KyiE9RI5F1fV60OIUu4",market="GB")